# Abstract

Read through a folder, importing everything, to do some basic searching about.

# Environment

In [1]:
from copy import copy
from importlib import import_module
from inspect import (getmembers, isclass)
import logging
from os import (listdir, path, walk)
import re
import sys

In [2]:
from jwst.stpipe import Step

In [25]:
SUFFIXES_TO_DISCARD = set(('functionwrapper', 'systemcall'))

In [37]:
SUFFIXES_TO_ADD = set((
    'cal', 'calints', 'crf', 'crfints',
    'dark',
    'i2d',
    'jump',
    'psfalign', 'psfstack', 'psfsub',
    'ramp', 'rate', 'rateints',
    's2d', 's3d',
    'uncal',
    'wfscmb',
    'x1d', 'x1dints',
))

# Library

In [30]:
def folder_traverse(folder_path, basename_regex='.+', path_exclude_regex='^$'):
    """Generator of full file paths for all files
    in a folder.
    
    Parameters
    ----------
    folder_path: str
        The folder to traverse
        
    basename_regex: str
        Regular expression that must match
        the `basename` part of the file path.
        
    path_exclude_regex: str
        Regular expression to exclude a path.
        
    Returns
    -------
    generator
        A generator, return the next file.
    """
    basename_regex = re.compile(basename_regex)
    path_exclude_regex = re.compile(path_exclude_regex)
    for root, dirs, files in walk(folder_path):
        if path_exclude_regex.search(root):
            continue
        for file in files:
            if basename_regex.match(file):
                yield path.join(root, file)

In [31]:
def load_local_pkg(fpath):
    """Generator producing all modules under fpath
    """
    package_fpath, package = path.split(fpath)
    package_fpath_len = len(package_fpath) + 1
    sys_path = copy(sys.path)
    sys.path.insert(0, package_fpath)
    try:
        for module_fpath in folder_traverse(
            fpath, basename_regex='[^_].+\.py$', path_exclude_regex='tests'
        ):
            folder_path, fname = path.split(module_fpath[package_fpath_len:])
            module_path = folder_path.split('/')
            module_path.append(path.splitext(fname)[0])
            module_path = '.'.join(module_path)
            try:
                module = import_module(module_path)
            except Exception:
                logging.warning('Cannot load module "{}"'.format(module_path))
            else:
                yield module
    except Exception as exception:
        logging.warning('Exception occurred: "{}'.format(exception))
    finally:
        sys.path = sys_path

In [38]:
def find_suffixes():
    """Find all possible suffixes from the jwst package
    
    Returns
    -------
    suffixes: set
        The set of all programmatically findable suffixes.
        
    Notes
    -----
    This will load all of the `jwst` package. Consider if this
    is worth doing dynamically or only as a utility to update
    a static list.
    """
    suffixes = set()
    
    jwst = import_module('jwst')
    jwst_fpath = path.split(jwst.__file__)[0]
    
    # First traverse the code base and find all
    # `Step` classes. The default suffix is the
    # class name.
    for module in load_local_pkg(jwst_fpath):
        for klass_name, klass in getmembers(
            module,
            lambda o: isclass(o) and issubclass(o, Step)
        ):
            suffixes.add(klass_name.lower())
            
    # Instantiate Steps/Pipelines from their configuration files.
    # Different names and suffixes can be defined in this way.
    # Note: Based on the `collect_pipeline_cfgs` script
    config_path = path.join(jwst_fpath, 'pipeline') 
    for config_file in listdir(config_path):
        if config_file.endswith('.cfg'):
            try:
                step = Step.from_config_file(path.join(config_path, config_file))
            except Exception as exception:
                pass
            else:
                suffixes.add(step.name.lower())
                if step.suffix is not None:
                    suffixes.add(step.suffix.lower())
                    
    # Discard known bad finds.
    suffixes.difference_update(SUFFIXES_TO_DISCARD)
    
    # Add defined suffixes
    suffixes.update(SUFFIXES_TO_ADD)
                    
    # That's all folks
    return suffixes

# Main

In [39]:
re.search('^$', '/Users/eisenham/Documents/ssbdev/jwst/jwst/associations/notebooks')

In [40]:
jwst = import_module('jwst')
jwst.__file__

'/Users/eisenham/Documents/ssbdev/jwst/jwst/__init__.py'

## Setup the all-in-one function

In [41]:
suffixes_from_find = find_suffixes()

2018-04-06 13:55:20,615 - stpipe - WARNING - Cannot load module "jwst.cube_build.CubeD2C"
2018-04-06 13:55:20,643 - stpipe - WARNING - Cannot load module "jwst.lib.convert_to_uncal"
2018-04-06 13:55:20,653 - stpipe - WARNING - Cannot load module "jwst.ramp_fitting.mc_2d"
2018-04-06 13:55:20,656 - stpipe - WARNING - Cannot load module "jwst.ramp_fitting.mc_3d"
2018-04-06 13:55:20,658 - stpipe - WARNING - Cannot load module "jwst.ramp_fitting.rotate_dataset"
2018-04-06 13:55:20,660 - stpipe - WARNING - Cannot load module "jwst.ramp_fitting.rotate_utils"
2018-04-06 13:55:20,667 - stpipe.alignrefs - INFO - AlignRefsStep instance created.
2018-04-06 13:55:20,669 - stpipe.ami_analyze - INFO - AmiAnalyzeStep instance created.
2018-04-06 13:55:20,671 - stpipe.ami_average - INFO - AmiAverageStep instance created.
2018-04-06 13:55:20,674 - stpipe.ami_normalize - INFO - AmiNormalizeStep instance created.
2018-04-06 13:55:20,676 - stpipe.assign_wcs - INFO - AssignWcsStep instance created.
2018-04-

2018-04-06 13:55:20,840 - stpipe.Spec3Pipeline.extract_1d - INFO - Extract1dStep instance created.
2018-04-06 13:55:20,850 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2018-04-06 13:55:20,852 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2018-04-06 13:55:20,854 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2018-04-06 13:55:20,855 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2018-04-06 13:55:20,857 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2018-04-06 13:55:20,859 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2018-04-06 13:55:20,860 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2018-04-06 13:55:20,861 - stpipe.Detector1Pipeline.rscd - INFO - RSCD_Step instance created.
2018-04-06 13:55:20,863 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2018-04-06 13:

In [42]:
len(suffixes_from_find)

137

In [43]:
suffixes_from_find

{'alignrefs',
 'alignrefsstep',
 'ami3pipeline',
 'ami_analyze',
 'ami_average',
 'ami_normalize',
 'amianalyzestep',
 'amiaveragestep',
 'aminormalizestep',
 'assign_wcs',
 'assignwcsstep',
 'background',
 'backgroundstep',
 'barshadowstep',
 'cal',
 'calints',
 'combine1dstep',
 'combine_1d',
 'coron3pipeline',
 'crf',
 'crfints',
 'cube_build',
 'cubebuildstep',
 'cubeskymatchstep',
 'dark',
 'dark_current',
 'darkcurrentstep',
 'darkpipeline',
 'detector1pipeline',
 'dq_init',
 'dqinitstep',
 'emission',
 'emissionstep',
 'engdblog',
 'engdblogstep',
 'extract1dstep',
 'extract2dstep',
 'extract_1d',
 'extract_2d',
 'firstframe',
 'firstframestep',
 'flat_field',
 'flatfieldstep',
 'fringe',
 'fringestep',
 'gain_scale',
 'gainscalestep',
 'group_scale',
 'groupscalestep',
 'guidercds',
 'guidercdsstep',
 'guiderpipeline',
 'hlsp',
 'hlspstep',
 'i2d',
 'image2pipeline',
 'image3pipeline',
 'imprint',
 'imprintstep',
 'ipc',
 'ipcstep',
 'jump',
 'jumpstep',
 'klip',
 'klipstep',
 

## The piecemeal developement

In [11]:
package_location = '../../../../jwst'

In [12]:
%ls $package_location

__init__.py        emission/          lib/               source_catalog/
__init__.pyc       engdblog/          linearity/         srctype/
__pycache__/       exp_to_source/     model_blender/     steps.py
ami/               extract_1d/        mrs_imatch/        stpipe/
assign_wcs/        extract_2d/        msaflagopen/       straylight/
associations/      firstframe/        outlier_detection/ superbias/
background/        fits_generator/    pathloss/          tests/
barshadow/         flatfield/         persistence/       timeconversion/
cal_ver_steps.py   fringe/            photom/            transforms/
combine_1d/        gain_scale/        pipeline/          tso_photometry/
coron/             group_scale/       ramp_fitting/      tweakreg/
csv_tools/         guider_cds/        refpix/            tweakreg_catalog/
cube_build/        imprint/           resample/          version.py
cube_skymatch/     ipc/               reset/             version.pyc
dark_current/      jump/           

## Start collecting all possible suffixes

In [13]:
suffixes = set()

## First, find all Step classes
`Step` class names are the default suffix for any `Step`

In [14]:
modules = list(load_local_pkg(package_location))

root="../../../../jwst"
root="../../../../jwst/__pycache__"
root="../../../../jwst/ami"
root="../../../../jwst/ami/__pycache__"
root="../../../../jwst/assign_wcs"
root="../../../../jwst/assign_wcs/__pycache__"
root="../../../../jwst/assign_wcs/tests"
	****rejected
root="../../../../jwst/assign_wcs/tests/__pycache__"
	****rejected
root="../../../../jwst/assign_wcs/tests/data"
	****rejected
root="../../../../jwst/assign_wcs/tests/data/__pycache__"
	****rejected
root="../../../../jwst/assign_wcs/tools"
root="../../../../jwst/assign_wcs/tools/__pycache__"
root="../../../../jwst/assign_wcs/tools/miri"
root="../../../../jwst/assign_wcs/tools/miri/__pycache__"
root="../../../../jwst/assign_wcs/tools/nirspec"
root="../../../../jwst/assign_wcs/tools/nirspec/__pycache__"
root="../../../../jwst/associations"
root="../../../../jwst/associations/__pycache__"
root="../../../../jwst/associations/lib"


2018-04-06 13:10:34,842 - stpipe - WARNING - Cannot load module "jwst.cube_build.CubeD2C"
2018-04-06 13:10:34,890 - stpipe - WARNING - Cannot load module "jwst.lib.convert_to_uncal"
2018-04-06 13:10:34,905 - stpipe - WARNING - Cannot load module "jwst.ramp_fitting.mc_2d"
2018-04-06 13:10:34,907 - stpipe - WARNING - Cannot load module "jwst.ramp_fitting.mc_3d"
2018-04-06 13:10:34,909 - stpipe - WARNING - Cannot load module "jwst.ramp_fitting.rotate_dataset"
2018-04-06 13:10:34,911 - stpipe - WARNING - Cannot load module "jwst.ramp_fitting.rotate_utils"


root="../../../../jwst/associations/lib/__pycache__"
root="../../../../jwst/associations/tests"
	****rejected
root="../../../../jwst/associations/tests/.ipynb_checkpoints"
	****rejected
root="../../../../jwst/associations/tests/__pycache__"
	****rejected
root="../../../../jwst/associations/tests/data"
	****rejected
root="../../../../jwst/associations/tests/data/.ipynb_checkpoints"
	****rejected
root="../../../../jwst/associations/tests/data/__pycache__"
	****rejected
root="../../../../jwst/associations/tests/data/asn_standards"
	****rejected
root="../../../../jwst/associations/tests/data/exposures"
	****rejected
root="../../../../jwst/associations/tests/data/new_asn_standards"
	****rejected
root="../../../../jwst/associations/tests/data/sdp"
	****rejected
root="../../../../jwst/associations/tests/data/sdp/asns"
	****rejected
root="../../../../jwst/associations/tests/data/sdp/new_asns"
	****rejected
root="../../../../jwst/associations/tests/data/sdp/pool_src"
	****rejected
root="../../.

In [15]:
step_klasses = set(
    klass
    for module in modules
    for klass_name, klass in getmembers(module, lambda o: isclass(o) and issubclass(o, Step))
)

In [16]:
step_klass_objs = []
for klass in step_klasses:
    try:
        klass_obj = klass()
    except Exception as exception:
        logging.warning('Cannot instantiate {}'.format(klass))
    else:
        step_klass_objs.append(klass_obj)


2018-04-06 13:10:35,018 - stpipe.Extract1dStep - INFO - Extract1dStep instance created.
2018-04-06 13:10:35,032 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2018-04-06 13:10:35,034 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2018-04-06 13:10:35,036 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2018-04-06 13:10:35,037 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2018-04-06 13:10:35,038 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2018-04-06 13:10:35,040 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2018-04-06 13:10:35,042 - stpipe.Spec2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2018-04-06 13:10:35,043 - stpipe.Spec2Pipeline.srctype - INFO - SourceTypeStep instance created.
2018-04-06 13:10:35,044 - stpipe.Spec2Pipeline.straylight - INFO - StraylightStep instance created.
2018-04-06 13:10:35,04

2018-04-06 13:10:35,234 - stpipe.AmiAverageStep - INFO - AmiAverageStep instance created.
2018-04-06 13:10:35,235 - stpipe.AmiNormalizeStep - INFO - AmiNormalizeStep instance created.
2018-04-06 13:10:35,236 - stpipe.RefPixStep - INFO - RefPixStep instance created.
2018-04-06 13:10:35,242 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2018-04-06 13:10:35,243 - stpipe.Spec3Pipeline.mrs_imatch - INFO - MRSIMatchStep instance created.
2018-04-06 13:10:35,245 - stpipe.Spec3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2018-04-06 13:10:35,247 - stpipe.Spec3Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2018-04-06 13:10:35,249 - stpipe.Spec3Pipeline.cube_build - INFO - CubeBuildStep instance created.
2018-04-06 13:10:35,252 - stpipe.Spec3Pipeline.extract_1d - INFO - Extract1dStep instance created.
2018-04-06 13:10:35,253 - stpipe.RampFitStep - INFO - RampFitStep instance created.
2018-04-06 13:10:35,254 - stpipe.SourceCatalogS

In [17]:
for step_klass_obj in step_klass_objs:
    try:
        name = step_klass_obj.name
    except Exception as exception:
        pass
    else:
        suffixes.add(name.lower())

In [18]:
len(suffixes)

69

## Instantiate from config files
`Step` and `Pipeline` can be instantiated from config files. These can define different names and suffixes. Collect what we can.

_Note_: Based on `collect_cfgs` script where configuration files are only searched for in the `pipeline` module.

In [19]:
config_path = path.join(package_location, 'pipeline')

In [20]:
for config_file in listdir(config_path):
    if config_file.endswith('.cfg'):
        try:
            step = Step.from_config_file(path.join(config_path, config_file))
        except Exception as exception:
            logging.warning('Could not instantiate {}'.format(config_file))
        else:
            suffixes.add(step.name.lower())
            if step.suffix is not None:
                suffixes.add(step.suffix.lower())

2018-04-06 13:10:35,560 - stpipe.alignrefs - INFO - AlignRefsStep instance created.
2018-04-06 13:10:35,562 - stpipe.ami_analyze - INFO - AmiAnalyzeStep instance created.
2018-04-06 13:10:35,564 - stpipe.ami_average - INFO - AmiAverageStep instance created.
2018-04-06 13:10:35,566 - stpipe.ami_normalize - INFO - AmiNormalizeStep instance created.
2018-04-06 13:10:35,567 - stpipe.assign_wcs - INFO - AssignWcsStep instance created.
2018-04-06 13:10:35,570 - stpipe.background - INFO - BackgroundStep instance created.
2018-04-06 13:10:35,576 - stpipe.Ami3Pipeline - INFO - Ami3Pipeline instance created.
2018-04-06 13:10:35,577 - stpipe.Ami3Pipeline.ami_analyze - INFO - AmiAnalyzeStep instance created.
2018-04-06 13:10:35,578 - stpipe.Ami3Pipeline.ami_average - INFO - AmiAverageStep instance created.
2018-04-06 13:10:35,579 - stpipe.Ami3Pipeline.ami_normalize - INFO - AmiNormalizeStep instance created.
2018-04-06 13:10:35,584 - stpipe.Coron3Pipeline - INFO - Coron3Pipeline instance created.


2018-04-06 13:10:35,753 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2018-04-06 13:10:35,755 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2018-04-06 13:10:35,756 - stpipe.Detector1Pipeline.rscd - INFO - RSCD_Step instance created.
2018-04-06 13:10:35,758 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2018-04-06 13:10:35,759 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2018-04-06 13:10:35,760 - stpipe.Detector1Pipeline.linearity - INFO - LinearityStep instance created.
2018-04-06 13:10:35,762 - stpipe.Detector1Pipeline.dark_current - INFO - DarkCurrentStep instance created.
2018-04-06 13:10:35,763 - stpipe.Detector1Pipeline.persistence - INFO - PersistenceStep instance created.
2018-04-06 13:10:35,765 - stpipe.Detector1Pipeline.jump - INFO - JumpStep instance created.
2018-04-06 13:10:35,767 - stpipe.Detector1Pipeline.ramp_fit - INFO - RampFitStep instance create

In [21]:
len(suffixes)

122

In [22]:
suffixes_from_find.symmetric_difference(suffixes)

{'functionwrapper', 'systemcall'}

In [23]:
suffixes

{'alignrefs',
 'alignrefsstep',
 'ami3pipeline',
 'ami_analyze',
 'ami_average',
 'ami_normalize',
 'amianalyzestep',
 'amiaveragestep',
 'aminormalizestep',
 'assign_wcs',
 'assignwcsstep',
 'background',
 'backgroundstep',
 'barshadowstep',
 'combine1dstep',
 'combine_1d',
 'coron3pipeline',
 'cube_build',
 'cubebuildstep',
 'cubeskymatchstep',
 'dark',
 'dark_current',
 'darkcurrentstep',
 'darkpipeline',
 'detector1pipeline',
 'dq_init',
 'dqinitstep',
 'emission',
 'emissionstep',
 'engdblog',
 'engdblogstep',
 'extract1dstep',
 'extract2dstep',
 'extract_1d',
 'extract_2d',
 'firstframe',
 'firstframestep',
 'flat_field',
 'flatfieldstep',
 'fringe',
 'fringestep',
 'gain_scale',
 'gainscalestep',
 'group_scale',
 'groupscalestep',
 'guidercds',
 'guidercdsstep',
 'guiderpipeline',
 'hlsp',
 'hlspstep',
 'i2d',
 'image2pipeline',
 'image3pipeline',
 'imprint',
 'imprintstep',
 'ipc',
 'ipcstep',
 'jump',
 'jumpstep',
 'klip',
 'klipstep',
 'lastframe',
 'lastframestep',
 'lineari